In [28]:
# # !gunzip cx-corpora.en2es.text.json.gz
# !pip install --user pybind11
# !pip install fasttext
#!conda install -c conda-forge fasttext

In [2]:
import json
import fasttext
import pandas as pd
import numpy as np
import re
import os
import unicodedata
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from numpy.core.umath_tests import inner1d

In [3]:
def apply_transform(vec, transform):
    """
    Apply the given transformation to the vector space

    Right-multiplies given transform with embeddings E:
        E = E * transform

    Transform can either be a string with a filename to a
    text file containing a ndarray (compat. with np.loadtxt)
    or a numpy ndarray.
    """
    transmat = np.loadtxt(transform)# if isinstance(transform, str) else transform
    return np.matmul(vec, transmat)

def make_training_matrices(source_dictionary, target_dictionary, bilingual_dictionary):
    """
    Source and target dictionaries are the FastVector objects of
    source/target languages. bilingual_dictionary is a list of 
    translation pair tuples [(source_word, target_word), ...].
    """
    source_matrix = []
    target_matrix = []
    
    len_bd = len(bilingual_dictionary)

    for i, (source, target) in tqdm(enumerate(bilingual_dictionary)):
#         print(f'\r{i + 1}/{len_bd} | {100 * (i + 1) / len_bd:.3f} %', end = '', flush = True)
        sourceVector = source_dictionary.get_sentence_vector(source.lower().strip().replace('_',' '))
        targetVector = target_dictionary.get_sentence_vector(target.lower().strip().replace('_',' '))
        source_matrix.append(sourceVector)
        target_matrix.append(targetVector)
        
    # return training matrices
    return np.array(source_matrix), np.array(target_matrix)

def normalized(a, axis=-1, order=2):
    """Utility function to normalize the rows of a numpy array."""
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)


def learn_transformation(source_matrix, target_matrix, normalize_vectors=True):
    """
    Source and target matrices are numpy arrays, shape
    (dictionary_length, embedding_dimension). These contain paired
    word vectors from the bilingual dictionary.
    """
    # optionally normalize the training vectors
    if normalize_vectors:
        source_matrix = normalized(source_matrix)
        target_matrix = normalized(target_matrix)

    # perform the SVD
    product = np.matmul(target_matrix.transpose(), source_matrix)
    U, s, V = np.linalg.svd(product)

    # return orthogonal transformation which aligns source language to the target
    return np.matmul(U, V)

def cleanData(data):
    data_cleaned = []
    for source, target in zip(data['source_value'], data['target_value']):
        try:
            source_cleaned = unicodedata.normalize('NFD', source)
            target_cleaned = unicodedata.normalize('NFD', target)
            if not source == ''  and not target == '':
                data_cleaned.append([source_cleaned, target_cleaned])
        except:
            pass
    return data_cleaned

In [5]:
model = {}
model['en'] = fasttext.load_model('cc.en.300.bin')
model['es'] = fasttext.load_model('cc.es.300.bin')

In [6]:
data = pd.read_csv('description_train.csv')

In [7]:
test = pd.read_csv('../ContentTranslation/content_test.csv') #same for everyone
train = pd.read_csv('description_train.csv') #replace with your respective dataset

In [8]:
#Train
bilingual_dictionary = list(zip(train['source'],train['target']))
source_matrix, target_matrix = make_training_matrices(model['en'], model['es'], bilingual_dictionary)

666867it [00:55, 11930.04it/s]


In [9]:
transform = learn_transformation(source_matrix, target_matrix)

In [10]:
# np.savetxt('description_trans_transform_en_es.txt', transform)

In [11]:
print("Before trans:", np.mean(inner1d(target_matrix, source_matrix)))

Before trans: 0.013913396191837644


In [12]:
print("After trans:", np.mean(inner1d(normalized(target_matrix), np.matmul(transform, normalized(source_matrix).T).T)))

After trans: 0.71027496082993


In [13]:
bilingual_dictionary = list(zip(test['source'],test['target']))
source_matrix_test, target_matrix_test = make_training_matrices(model['en'], model['es'], bilingual_dictionary)

72972it [00:50, 1445.54it/s]


In [14]:
#before
target_matrix_test = normalized(target_matrix_test)
source_matrix_test = normalized(source_matrix_test)
print("Before trans:",np.mean(inner1d(target_matrix_test, source_matrix_test)))
#after
print("After trans:", np.mean(inner1d(target_matrix_test, np.matmul(transform, source_matrix_test.T).T)))

Before trans: 0.04509296739724158
After trans: 0.7376235840350941
